In [23]:
import pandas as pd


In [24]:

file_path = "E2.csv"

data = pd.read_csv(file_path)


print(data.head())


  Div        Date   Time      HomeTeam     AwayTeam  FTHG  FTAG FTR  HTHG  \
0  E2  09/08/2024  20:00      Barnsley    Mansfield     1     2   A     1   
1  E2  10/08/2024  17:30    Birmingham      Reading     1     1   D     0   
2  E2  10/08/2024  17:30   Bristol Rvs  Northampton     1     0   H     0   
3  E2  10/08/2024  17:30        Burton      Lincoln     2     3   A     2   
4  E2  10/08/2024  17:30  Crawley Town    Blackpool     2     1   H     2   

   HTAG  ... B365CAHH B365CAHA  PCAHH  PCAHA  MaxCAHH  MaxCAHA  AvgCAHH  \
0     2  ...     2.03     1.83   2.02   1.87     2.05     2.00     1.95   
1     1  ...     1.80     2.05   1.81   2.10     1.83     2.10     1.80   
2     0  ...     1.98     1.88   1.98   1.91     1.98     1.92     1.93   
3     2  ...     1.80     2.05   1.82   2.08     1.88     2.09     1.80   
4     0  ...     1.83     2.03   1.85   2.04     1.86     2.05     1.82   

   AvgCAHA  BFECAHH  BFECAHA  
0     1.85     2.06     1.93  
1     2.01     1.82     

In [ ]:
outcome_counts = data['FTR'].value_counts()
total_matches = len(data)
outcome_percentages = (outcome_counts / total_matches) * 100

print("Match Outcome Distribution:")
print(outcome_counts)
print("Outcome Percentages (%):")
print(outcome_percentages)

Match Outcome Distribution:
FTR
H    86
A    56
D    44
Name: count, dtype: int64
Outcome Percentages (%):
FTR
H    46.236559
A    30.107527
D    23.655914
Name: count, dtype: float64


In [37]:
home_wins = data[data['FTR'] == 'H']
away_wins = data[data['FTR'] == 'A']
draws = data[data['FTR'] == 'D']


avg_home_odds = home_wins['B365H'].mean()
avg_away_odds = away_wins['B365A'].mean()
avg_draw_odds = draws['B365D'].mean() 

print("\nAverage Odds for Each Outcome:")
print(f"Home Win: {avg_home_odds:.2f}")
print(f"Away Win: {avg_away_odds:.2f}")
print(f"Draw: {avg_draw_odds:.2f}")


Average Odds for Each Outcome:
Home Win: 2.19
Away Win: 2.94
Draw: 3.61


In [32]:
home_upsets = home_wins[home_wins['B365H'] > 3.0]
away_upsets = away_wins[away_wins['B365A'] > 3.0]

upset_count = len(home_upsets) + len(away_upsets)
upset_percentage = (upset_count / total_matches) * 100

print(f"\nNumber of Upsets (Odds > 3.0): {upset_count}")
print(f"Upset Percentage: {upset_percentage:.2f}%")


Number of Upsets (Odds > 3.0): 29
Upset Percentage: 15.59%


In [ ]:
upsets = pd.concat([
    home_upsets[['HomeTeam', 'AwayTeam', 'B365H', 'FTR', 'Date']], 
    away_upsets[['HomeTeam', 'AwayTeam', 'B365A', 'FTR', 'Date']]
])



# Add a column for odds, which will be the home odds if it's a home upset, or away odds if it's an away upset
upsets['Odds'] = upsets['B365H'].fillna(upsets['B365A'])

# Add a column to indicate if the odds are for home or away
upsets['OddsType'] = upsets.apply(
    lambda row: 'Home Odds' if pd.notna(row['B365H']) else 'Away Odds', axis=1
)

# Sort the upsets by the odds in descending order
upsets_sorted = upsets.sort_values(by='Odds', ascending=False)

# Select the top 5 upsets
top_5_upsets = upsets_sorted.head(5)

# Display the top 5 upsets
print("\nTop 5 Biggest Upsets (Odds > 3.0):")
for index, row in top_5_upsets.iterrows():
    print(f"Date: {row['Date']}, Home Team: {row['HomeTeam']}, Away Team: {row['AwayTeam']}, "
          f"{row['OddsType']}: {row['Odds']:.2f}, Full-Time Result: {row['FTR']}")

         HomeTeam       AwayTeam  B365H FTR        Date  B365A
4    Crawley Town      Blackpool    3.4   H  10/08/2024    NaN
106      Charlton     Birmingham    4.0   H  05/10/2024    NaN
121     Cambridge          Wigan    3.2   H  19/10/2024    NaN
126   Northampton  Leyton Orient    3.1   H  19/10/2024    NaN
132  Crawley Town        Lincoln    3.4   H  22/10/2024    NaN

Top 5 Biggest Upsets (Odds > 3.0):
Date: 21/09/2024, Home Team: Huddersfield, Away Team: Northampton, Away Odds: 8.00, Full-Time Result: A
Date: 23/11/2024, Home Team: Shrewsbury, Away Team: Birmingham, Home Odds: 7.50, Full-Time Result: H
Date: 31/08/2024, Home Team: Bolton, Away Team: Exeter, Away Odds: 6.50, Full-Time Result: A
Date: 21/09/2024, Home Team: Stockport, Away Team: Leyton Orient, Away Odds: 6.00, Full-Time Result: A
Date: 22/10/2024, Home Team: Stevenage, Away Team: Cambridge, Away Odds: 5.00, Full-Time Result: A


In [42]:
# Identify home upsets (home wins with odds > 3.0)
home_upsets = home_wins[home_wins['B365H'] > 3.0]

# Sort the home upsets by the home odds in descending order
home_upsets_sorted = home_upsets[['HomeTeam', 'AwayTeam', 'B365H', 'FTR', 'Date']].sort_values(by='B365H', ascending=False)

# Select the top 5 home upsets
top_5_home_upsets = home_upsets_sorted.head(5)

# Display the top 5 home upsets with clear indication of home odds
print("\nTop 5 Biggest Home Upsets (Home Team Wins with Odds > 3.0):")
for index, row in top_5_home_upsets.iterrows():
    print(f"Date: {row['Date']}, Home Team: {row['HomeTeam']}, Away Team: {row['AwayTeam']}, "
          f"Home Odds: {row['B365H']:.2f}, Full-Time Result: {row['FTR']}")



Top 5 Biggest Home Upsets (Home Team Wins with Odds > 3.0):
Date: 23/11/2024, Home Team: Shrewsbury, Away Team: Birmingham, Home Odds: 7.50, Full-Time Result: H
Date: 05/10/2024, Home Team: Charlton, Away Team: Birmingham, Home Odds: 4.00, Full-Time Result: H
Date: 23/11/2024, Home Team: Crawley Town, Away Team: Rotherham, Home Odds: 3.60, Full-Time Result: H
Date: 22/10/2024, Home Team: Crawley Town, Away Team: Lincoln, Home Odds: 3.40, Full-Time Result: H
Date: 10/08/2024, Home Team: Crawley Town, Away Team: Blackpool, Home Odds: 3.40, Full-Time Result: H


In [44]:
# Calculate favorite win percentage
data['Favorite'] = data[['B365H', 'B365D', 'B365A']].idxmin(axis=1)
data['FavoriteWon'] = ((data['Favorite'] == 'B365H') & (data['FTR'] == 'H')) | \
                      ((data['Favorite'] == 'B365A') & (data['FTR'] == 'A'))

# Calculate the percentage of matches where the favorite won
favorite_win_percentage = (data['FavoriteWon'].sum() / total_matches) * 100

# Calculate the percentage of matches that ended in a draw
draw_percentage = (data['FTR'] == 'D').sum() / total_matches * 100

# Calculate the percentage of matches where the favorite lost (i.e., favorite not winning)
favorite_loss_percentage = ((data['FavoriteWon'] == False) & (data['FTR'] != 'D')).sum() / total_matches * 100

# Display the results
print(f"\nPercentage of Matches Where the Favorite Won: {favorite_win_percentage:.2f}%")
print(f"Percentage of Matches that Were a Draw: {draw_percentage:.2f}%")
print(f"Percentage of Matches Where the Favorite Lost: {favorite_loss_percentage:.2f}%")



Percentage of Matches Where the Favorite Won: 51.61%
Percentage of Matches that Were a Draw: 23.66%
Percentage of Matches Where the Favorite Lost: 24.73%
